In [476]:
import import_ipynb
from Functions import *
import torch.nn.functional as Fun
import torch
from torch import optim
import matplotlib.pyplot as plt
import time
from torch import nn
from torchvision import transforms

# Add Inconsistency

In [477]:
def AddOneInc1(F):
    # Convert to NumPy Array
    F = F.squeeze(0).permute(1, 2, 0).cpu().numpy() * 255
    # Copy of original array
    I = F.astype(np.uint8).copy()
    # Random location
    x, y = random.randint(0, 2 * I.shape[0] // 3), random.randint(0, 2 * I.shape[1] // 3)
    # Random size
    l = random.randint(5, I.shape[1] // 20)
    # Random option
    Op = random.randint(0, 5)
    if Op == 0:
        I[x:x + l, y:y + l] = change_range_colors(
            I[x:x + l, y:y + l],
            (random.randint(30, 140), random.randint(30, 140), random.randint(30, 140)),
            (random.randint(150, 255), random.randint(150, 255), random.randint(150, 255))
        )
    elif Op == 1:
        R, G, B = random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)
        I[x:x + l, y:y + l] = change_range_colors(I[x:x + l, y:y + l], (R, G, B), (R, G, B))
    elif Op == 2:
        Thick = random.randint(1, 10)
        l2 = random.randint(5, I.shape[1] // 15)
        I = cv2.line(I, (x, y), (x + l, y + l2), (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)), Thick)
    elif Op == 3:
        Thick = random.randint(1, 5)
        l2 = random.randint(5, I.shape[1] // 20)
        I = cv2.line(I, (x, y), (x + l, y + l2), (0, 0, 0), Thick)
    elif Op == 4:
        I[x:x + l, y:y + l] = I[x:x + l, y:y + l] + np.random.randint(-10, 10, I[x:x + l, y:y + l].shape)
    else:
        kernel = np.random.rand(3, 3)
        I[x:x + l, y:y + l] = cv2.filter2D(I[x:x + l, y:y + l], -1, kernel)
    
    # Convert back to tensor
    I_tensor = torch.tensor(I, dtype=torch.float32).permute(2, 0, 1) / 255.0
    return I_tensor

# Calculate OF

In [478]:
def OF(img1: torch.Tensor, img2: torch.Tensor, alpha: float = 1.0, iterations: int = 100):
    assert img1.shape == img2.shape, "Images must have the same shape"

    def rgb_to_grayscale(img):
        return 0.2989 * img[0, :, :] + 0.5870 * img[1, :, :] + 0.1140 * img[2, :, :]

    I1 = rgb_to_grayscale(img1)
    I2 = rgb_to_grayscale(img2)

    # Initialize optical flow vectors (u for x direction, v for y direction)
    u = torch.zeros_like(I1, requires_grad=True)
    v = torch.zeros_like(I1, requires_grad=True)
    
    # Define convolution kernels for gradients
    kernel_x = torch.tensor([[[[-1, 1], [-1, 1]]]], dtype=torch.float32)
    kernel_y = torch.tensor([[[[-1, -1], [1, 1]]]], dtype=torch.float32)

    # Compute gradients with padding that maintains the original image size
    Ix = Fun.conv2d(I1.unsqueeze(0).unsqueeze(0), kernel_x, padding=(0, 1)).squeeze(0).squeeze(0)
    Iy = Fun.conv2d(I1.unsqueeze(0).unsqueeze(0), kernel_y, padding=(1, 0)).squeeze(0).squeeze(0)
    It = I2 - I1  # Temporal gradient

    # Ensure all tensors have matching dimensions
    min_h = min(Ix.shape[-2], Iy.shape[-2], It.shape[-2], I1.shape[-2])
    min_w = min(Ix.shape[-1], Iy.shape[-1], It.shape[-1], I1.shape[-1])

    Ix = Ix[:min_h, :min_w]
    Iy = Iy[:min_h, :min_w]
    It = It[:min_h, :min_w]
    u = u[:min_h, :min_w]
    v = v[:min_h, :min_w]

    # Iteratively update the optical flow
    for _ in range(iterations):
        u_avg = Fun.avg_pool2d(u.unsqueeze(0).unsqueeze(0), 3, stride=1, padding=1).squeeze(0).squeeze(0)
        v_avg = Fun.avg_pool2d(v.unsqueeze(0).unsqueeze(0), 3, stride=1, padding=1).squeeze(0).squeeze(0)
        
        P = Ix * u_avg + Iy * v_avg + It
        D = alpha ** 2 + Ix ** 2 + Iy ** 2
        
        u = u_avg - (Ix * P) / D
        v = v_avg - (Iy * P) / D

    flow = torch.stack((u, v), dim=0)
    return flow

# Calculate Difference in Frequency

In [479]:
def frequency_loss(image1, image2):
    freq_image1 = torch.fft.fft2(image1 * 255, dim=(-2, -1))
    freq_image2 = torch.fft.fft2(image2 * 255, dim=(-2, -1))
    mag_image1 = torch.abs(freq_image1)
    mag_image2 = torch.abs(freq_image2)
    loss = mag_image1 - mag_image2
    return loss

# Main Model

In [480]:
class PixelModel(nn.Module):
    def __init__(self, N):
        super(PixelModel, self).__init__()
        self.N = N
        self.conv1 = nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 37 * 37, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.pool(Fun.relu(self.conv1(x)))
        x = self.pool(Fun.relu(self.conv2(x)))
        x = self.pool(Fun.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = Fun.relu(self.fc1(x))
        x = Fun.relu(self.fc2(x))
        x = self.fc3(x)
        return torch.mean(x)

In [481]:
class MixModel(nn.Module):
    def __init__(self, N):
        super(MixModel, self).__init__()
        self.N = N
        self.layer1 = nn.Linear(N, 64)
        self.layer2 = nn.Linear(64, 32)
        self.layer3 = nn.Linear(32, 16)
        self.output = nn.Linear(16, 1)
        
    def forward(self, x):
        x = Fun.relu(self.layer1(x))
        x = Fun.relu(self.layer2(x))
        x = Fun.relu(self.layer3(x))
        x = self.output(x)
        return Fun.sigmoid(x)

# Get Training Data

In [482]:
def get_video_frames(path, N, size):
    # Find all video files in the specified path
    video_files = [f for f in os.listdir(path) if f.endswith(('.mp4', '.avi', '.mov', '.mkv'))]
    if not video_files:
        raise ValueError("No video files found in the specified path.")
    
    # Choose a random video file
    video_file = random.choice(video_files)
    video_path = os.path.join(path, video_file)
    
    # Initialize the video capture
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # If N is larger than the number of frames in the video, adjust it
    if N > total_frames:
        raise ValueError(f"The video has only {total_frames} frames, but {N} frames were requested.")
    
    # Select a random starting frame index such that we can capture N consecutive frames
    start_frame = random.randint(0, total_frames - N)
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
    
    # Resize transformation
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(size),
        transforms.ToTensor()
    ])
    
    frames = []
    for _ in range(N):
        ret, frame = cap.read()
        if not ret:
            break
        # Convert the frame (BGR to RGB) and apply the resize transform
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_tensor = transform(frame_rgb)
        frames.append(frame_tensor)
    
    cap.release()
    
    # Stack frames and reshape to the desired output shape
    frames_tensor = torch.stack(frames).unsqueeze(0)  # Shape (1, N, 3, H, W)
    
    return frames#_tensor

# Convert Tensor to Numpy List

In [483]:
def t2nl(frames_tensor):
    # Remove the batch dimension (1, N, 3, H, W) -> (N, 3, H, W)
    frames_tensor = frames_tensor.squeeze(0)
    
    # Convert each frame tensor to a NumPy array
    frame_list = [cv2.cvtColor(frame.permute(1, 2, 0).numpy(),cv2.COLOR_BGR2RGB) for frame in frames_tensor]
    
    return frame_list

# Training Function

In [484]:
def train(Path, State=None, N=3, Batch=10, Epochs=10, Steps=5, LR=1e-3):
    pmodel = PixelModel(N)
    dmodel = PixelModel(N-1)
    fmodel = PixelModel(N-1)
    mmodel = MixModel(3)
    
    Params = list(pmodel.parameters())+list(dmodel.parameters())+list(fmodel.parameters())+list(mmodel.parameters())
    
    Size = (300,300)
    
    if State is not None:
        model.load_state_dict(State)

    Loss= []
    LMin = 1e20

    for epoch in range(Epochs):
        optimizer = optim.Adam(Params, lr=LR)
        total_loss = 0.0
        for batch in range(Batch):
            optimizer.zero_grad()
            
            Frames = get_video_frames(Path, N, Size)
            Frames = torch.stack(Frames)
            #print('Frames: ',Frames.shape)
            Target = torch.tensor([0.0]) if random.random() >= 0.5 else torch.tensor([1.0])

            if Target.item() == 1.0:
                steps = random.randint(1, Steps)
                for s in range(steps):
                    Frames[N // 2] = AddOneInc1(Frames[N // 2])
            
            # Difference Frames
            diff_frames = [Frames[i+1] - Frames[i] for i in range(len(Frames)-1)]
            diff_frames = torch.stack(diff_frames)  # Shape [1, N-1, C, H, W]
            #print('Diff: ',diff_frames.shape)
            
            #Frequency Frames
            freq_frames = [frequency_loss(Frames[i+1],Frames[i]) for i in range(len(Frames)-1)]
            freq_frames = torch.stack(freq_frames)
            #print('Freq: ',freq_frames.shape)
            
            # Forward pass
            predP = pmodel(Frames).unsqueeze(0)
            predD = dmodel(diff_frames).unsqueeze(0)
            predF = fmodel(freq_frames).unsqueeze(0)
            predAll = torch.stack([predP,predD,predF]).permute(1,0).squeeze(0)
            #print('All: ',predAll.shape)
            
            Pred = mmodel(predAll)
            #print('Pred: ',Pred.shape)
            #print('Target: ',Target.shape)
            # Calculate loss
            criterion = nn.BCEWithLogitsLoss()
            loss = criterion(Pred, Target)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            print(f'LR: {LR:0.4} Batch [{batch+1}/{Batch}], Loss: {loss.item():.6f}', end='\r')

            # Display predictions on the frame
            Frame = cv2.cvtColor(Frames[N // 2].permute(1,2,0).numpy(), cv2.COLOR_BGR2RGB)
            Frame = cv2.putText(Frame, f'P: {round(Pred.item())}', (0, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 1)
            Frame = cv2.putText(Frame, f'T: {int(Target.item())}', (0, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 1)
            cv2.imshow("Frame", Frame)
            cv2.waitKey(1)

        Loss.append(total_loss)
        print(f'\nEpoch [{epoch + 1}/{Epochs}], Loss: {total_loss / Batch:.6f}\n')
        
        #if total_loss < LMin:
         #   LMin = total_loss
          #  State = model.state_dict()
        #else:
         #   model.load_state_dict(State)
        LR *= 0.9
    
    cv2.destroyAllWindows()
            
    return State,pmodel,Loss

# Training

In [488]:
cv2.destroyAllWindows()

In [486]:
State = None

In [487]:
State,Model,L = train('VDB',Batch=10,State=State,N=3,Epochs=50,Steps=10,LR=1e-3)

LR: 0.001 Batch [10/10], Loss: 1.313262
Epoch [1/50], Loss: 0.828638

LR: 0.0009 Batch [10/10], Loss: 1.313262
Epoch [2/50], Loss: 0.813262

LR: 0.00081 Batch [10/10], Loss: 1.313262
Epoch [3/50], Loss: 1.113262

LR: 0.000729 Batch [10/10], Loss: 0.313262
Epoch [4/50], Loss: 0.913262

LR: 0.0006561 Batch [10/10], Loss: 0.313262
Epoch [5/50], Loss: 0.813262

LR: 0.0005905 Batch [10/10], Loss: 1.313262
Epoch [6/50], Loss: 0.913262

LR: 0.0005314 Batch [10/10], Loss: 0.313262
Epoch [7/50], Loss: 0.613262

LR: 0.0004783 Batch [10/10], Loss: 1.313262
Epoch [8/50], Loss: 0.813262

LR: 0.0004305 Batch [10/10], Loss: 0.313262
Epoch [9/50], Loss: 0.913262

LR: 0.0003874 Batch [10/10], Loss: 1.313262
Epoch [10/50], Loss: 0.913262

LR: 0.0003487 Batch [10/10], Loss: 1.313262
Epoch [11/50], Loss: 0.913262

LR: 0.0003138 Batch [10/10], Loss: 1.313262
Epoch [12/50], Loss: 0.813262

LR: 0.0002824 Batch [10/10], Loss: 1.313262
Epoch [13/50], Loss: 0.713262

LR: 0.0002542 Batch [10/10], Loss: 1.313262


KeyboardInterrupt: 

In [ ]:
plt.plot(L)

In [ ]:
def Prediction(video, model, model_state, N, H, W):
    # Load the model state
    model.load_state_dict(model_state)
    model.eval()  # Set model to evaluation mode
    outputs = []
    # Iterate over the video in steps of N frames
    num_frames = len(video)
    for i in range(0, num_frames - N + 1):
        # Extract the stack of frames
        frame_stack = video[i:i + N]  # Shape: (N, H, W, C)
        # Convert to numpy array and add batch dimension
        frame_stack = np.array(frame_stack)  # Shape: (N, H, W, C)
        # Rearrange to match the model's expected input shape: (batch, N, C, H, W)
        # In our case, the frame_stack shape should be (1, N, 3, H, W) where 3 is the number of channels (RGB)
        frame_stack = np.transpose(frame_stack, (0, 3, 1, 2))  # Shape: (N, C, H, W)
        frame_stack = np.expand_dims(frame_stack, axis=0)  # Shape: (1, N, C, H, W)
        # Convert to a PyTorch tensor
        frame_stack = torch.tensor(frame_stack, dtype=torch.float32)
        # Forward pass through the model
        with torch.no_grad():
            output = model(frame_stack)
        # Append the output to the list
        outputs.append(int(round(output.item())))
    return outputs

In [ ]:
cap = open_vid("AI Gen/movie.mp4")
Video = get_frames(cap)
VI = torch.from_numpy(Video[5]).permute(2,0,1).unsqueeze(0)
VI = AddOneInc1(VI)
Video[5] = cv2.cvtColor(VI.permute(1, 2, 0).numpy(),cv2.COLOR_BGR2RGB)
display_frame(Video[5])

In [ ]:
Video = [cv2.resize(i,(100,100)) for i in Video[:15]]
Preds = process_video(Video,Model,State,3,100,100)
Preds

In [ ]:
def Prediction(Vid,Model,State,N,Size):
    model = BinIncModel(N,Size(0),Size(1))
    P = []
    V = [torch.tensor(cv2.resize(i,(320,320),interpolation=cv2.INTER_AREA),
                              dtype=torch.float32).permute(2,0,1).unsqueeze(0)/255.0 for i in Vid]
    
    N = len(kt)//2
    for f in range(len(V) - 2 * N):
        first, current, last = f, f + N // 2, f + N
        Pred = ObjFun(V[first:last], V[current], k, kt, kb)
        P.append(cv2.resize((Pred[0] * 255).squeeze(0).permute(1, 2, 0).detach().cpu().numpy().astype(np.uint8),
                                           (Vid[f].shape[1], Vid[f].shape[0]), interpolation=cv2.INTER_AREA))
    return P

In [ ]:
PVid = Prediction(Video,Parameters)

In [ ]:
ThroughFrames(Video)

In [ ]:
ThroughFrames(PVid)

In [ ]:
##Add detail error, MSE of full image only minimizes general pixel distribution
##Add more training examples
##Add N previous and N post current inconsistence image
##Add optical flow, frequencies or other features for temporal consistency
##Different kernel sizes
##Add Gaisian Noise
##Reconstruct image with kernel middle between encoder and decofer
##Use data with less loss value, selection
##How to combine data with less error and with more error?
##Localize area and correct error

In [ ]:
#deformaciones
#parches de colores, OF ventana
#self attention mechanism
#image processing transformer